In [1]:
import os

os.chdir("../")

%pwd

'c:\\Users\\NgLaam\\Desktop\\chatbot\\text-summarization'

In [2]:
# # if you encounter error "You need to have sentencepiece installed to convert a slow tokenizer to a fast one"
# ! pip install transformers[sentencepiece] 
# # and restart the kernel

In [3]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Path

In [4]:
from text_summarizer.constants import *
from text_summarizer.utils.common import read_yaml, create_directories

In [5]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            tokenizer_name = config.tokenizer_name
        )

        return data_transformation_config

In [6]:
import pandas as pd
import os
from text_summarizer.logging import logger
from transformers import AutoTokenizer
from datasets import DatasetDict, Dataset

c:\Users\NgLaam\Desktop\chatbot\text-summarization\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2024-05-11 08:12:33,684: INFO: config: PyTorch version 2.3.0 available.]


In [7]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)


    
    def convert_examples_to_features(self,example_batch):
        input_encodings = self.tokenizer(example_batch['dialogue'] , max_length = 1024, truncation = True )
        
        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(example_batch['summary'], max_length = 128, truncation = True )
            
        return {
            'input_ids' : input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'labels': target_encodings['input_ids']
        }
    
    def split_dataset(self, df):
        # Chuyển đổi dữ liệu thành định dạng Dataset
        dataset_dict = DatasetDict()

        # Tách dữ liệu thành tập huấn luyện, tập kiểm tra và tập validation
        train_df = df.sample(frac=0.8, random_state=42)
        val_test_df = df.drop(train_df.index)
        val_df = val_test_df.sample(frac=0.5, random_state=42)
        test_df = val_test_df.drop(val_df.index)

        # Tạo Dataset cho mỗi tập
        train_dataset = Dataset.from_pandas(train_df.rename(columns={"Unnamed: 0": "id"}))
        val_dataset = Dataset.from_pandas(val_df.rename(columns={"Unnamed: 0": "id"}))
        test_dataset = Dataset.from_pandas(test_df.rename(columns={"Unnamed: 0": "id"}))

        # Gán features
        train_dataset = train_dataset.map(lambda example: {'id': example['id'], 'dialogue': example['Document'], 'summary': example['Summary']})
        val_dataset = val_dataset.map(lambda example: {'id': example['id'], 'dialogue': example['Document'], 'summary': example['Summary']})
        test_dataset = test_dataset.map(lambda example: {'id': example['id'], 'dialogue': example['Document'], 'summary': example['Summary']})

        # Đưa các dataset vào DatasetDict
        dataset_dict["train"] = train_dataset
        dataset_dict["validation"] = val_dataset
        dataset_dict["test"] = test_dataset

        return dataset_dict


    def convert(self):
        df = pd.read_csv(self.config.data_path)
        dataset_dict = self.split_dataset(df)
        dataset_samsum_pt = dataset_dict.map(self.convert_examples_to_features, batched = True)
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir,"summarize_dataset"))

In [8]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.convert()
except Exception as e:
    raise e

[2024-05-11 08:12:34,094: INFO: common: yaml file: config\config.yaml load successfully]
[2024-05-11 08:12:34,102: INFO: common: yaml file: params.yaml load successfully]
[2024-05-11 08:12:34,105: INFO: common: Created directory: artifacts]
[2024-05-11 08:12:34,109: INFO: common: Created directory: artifacts/data_transformation]


c:\Users\NgLaam\Desktop\chatbot\text-summarization\.venv\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Map:   0%|          | 0/8522 [00:00<?, ? examples/s]c:\Users\NgLaam\Desktop\chatbot\text-summarization\.venv\lib\site-packages\transformers\tokenization_utils_base.py:3921: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Saving the dataset (1/1 shards): 100%|██████████| 1065/1065 [00:00<00:00, 36049.53 examples/s]
